In [21]:
# imports
from pathlib import Path

from ipywidgets import IntProgress, Label, VBox
from IPython.display import display

In [14]:
# variables
root_path = Path('/Users/dlisla/Pictures/athletics_programs/Output/')

input_dir_path = root_path.joinpath('0.jp2_toValidate')
error_dir_path = root_path.joinpath('0a.jp2_errors')
valid_dir_path = root_path.joinpath('1.jp2_toQC')

# jHove comamand and module
jhove = '~/jhove/jhove'  # jHove location
jp2_module = '-m JPEG2000-hul'
tif_module = '-m  TIFF-hul'

In [28]:
# functions

# def batch_process(root_path):
#     dir_path_list = sorted([x for x in root_path.iterdir() if x.is_dir()])
#     for dir_path in dir_path_list:
#         print(f'processing {dir_path} . . . ')
#         errors = jhove_jp2(dir_path)
#         if errors:
#             print(f'errors: {errors}\n')
#         else:
#             print('No errors found\n')

def delete_dot_files(directory):
    dir_path = Path(directory)
    dot_file_path_list = list(dir_path.glob('**/.*'))
    print(f'Number of dot files before delete: {len(dot_file_path_list)}')
    for dot_file_path in dot_file_path_list:
        dot_file_path.unlink()
    dot_file_path_list = list(dir_path.glob('**/.*'))
    print(f'After delete: {len(dot_file_path_list)}')

    
def jhove_validate(directory):
    
    dir_path = Path(directory)
    print(f'jHove validating JP2s in {dir_path}')
    
    # get list of jp2 files & delete all .*.jp2 dot files
    jp2_path_list = sorted(dir_path.glob('**/*.jp2'))
    delete_dot_files(dir_path)
    jp2_path_list = sorted(dir_path.glob('**/*.jp2'))
    number_of_jp2 = len(jp2_path_list)
    print(f'Processing ---> {number_of_jp2} *.jp2 <--- in {dir_path}')
    
    # progress bar
    progress_label = Label('jHove verify images')
    progress_bar = IntProgress(min=0, max=number_of_jp2)
    progress_widget = VBox([progress_label, progress_bar])
    display(progress_widget)
    
    # loop through images
    for i, jp2_path in enumerate(jp2_path_list):
        
        label = f'jHove validating {jp2_path.name}'
        progress_label.value = label
        
        # get identifier file list
        identifier_path_list = list(jp2_path.parent.glob(f'{jp2_path.stem}.*'))
        #print(identifier_path_list)
        
        # every 10 images print out an update
#         if i % 10 == 0:
#             print(f'processing image: {jp2_path.name}\nimage {i+1} of {number_of_jp2}')
        
        # validate image
        status = !{jhove} {jp2_module} "{str(jp2_path)}" | grep -e 'Status:'
        status = status[0]  # get first item in list
        # print(status)
        # set output directory path
        if status == '  Status: Well-Formed and valid':
            output_dir_path = valid_dir_path
                
        else:  # not valid
            output_dir_path = error_dir_path
        
        # move identifier files
        for identifier_path in identifier_path_list:
            new_identifier_path = output_dir_path.joinpath(identifier_path.name)
            identifier_path.rename(new_identifier_path)
        
        progress_bar.value = i + 1
            
        



In [29]:
jhove_validate(input_dir_path)

jHove validating JP2s in /Users/dlisla/Pictures/athletics_programs/Output/0.jp2_toValidate
Number of dot files before delete: 0
After delete: 0
Processing ---> 710 *.jp2 <--- in /Users/dlisla/Pictures/athletics_programs/Output/0.jp2_toValidate


In [6]:
!open {str(root_path)}